In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("BlastocystData_Image.csv")
silver = pd.read_csv("silver_labeled.csv")
gold = pd.read_csv("gold_labeled.csv")

In [6]:
silver["category"] = "Silver"

In [7]:
gold["category"] = "Gold"

In [8]:
#Renaming columns for silver training set
silver = silver.rename(columns={"EXP_silver": "EXP", "ICM_silver": "ICM", "TE_silver": "TE"})

#Renaming columns for gold testing set
gold = gold.rename(columns={"EXP_gold": "EXP", "ICM_gold": "ICM", "TE_gold": "TE"})

In [11]:
#make one big train test file with corresponding silver/gold label for train, test indication
result = pd.concat([silver, gold])

In [17]:
#create one dataset with corresponding labels and cell tool features
relevant_categories = result[["Image", "label", "category"]]

def filter_string(string):
    #change file path below to the file path shown in your df["URL_BlastocystOriginal"]
    returnval = string.replace("file:/Users/ElizabethMiriya/Downloads/Blastocyst_Dataset/Images/", "")
    return returnval

df["URL_BlastocystOriginal"] = df["URL_BlastocystOriginal"].apply(lambda x: filter_string(x))
relevant_categories = relevant_categories.rename(columns={"Image": "URL_BlastocystOriginal"})
aligned_df = pd.merge(df, relevant_categories, on='URL_BlastocystOriginal', how='inner') 

In [19]:
#drop columns with too many NA values
aligned_df.drop(columns = ['Mean_ICM_AreaShape_NormalizedMoment_0_0',
       'Mean_ICM_AreaShape_NormalizedMoment_0_1',
       'Mean_ICM_AreaShape_NormalizedMoment_1_0',
       'Mean_Outlines_AreaShape_NormalizedMoment_0_0',
       'Mean_Outlines_AreaShape_NormalizedMoment_0_1',
       'Mean_Outlines_AreaShape_NormalizedMoment_1_0',
       'Median_ICM_AreaShape_NormalizedMoment_0_0',
       'Median_ICM_AreaShape_NormalizedMoment_0_1',
       'Median_ICM_AreaShape_NormalizedMoment_1_0',
       'Median_Outlines_AreaShape_NormalizedMoment_0_0',
       'Median_Outlines_AreaShape_NormalizedMoment_0_1',
       'Median_Outlines_AreaShape_NormalizedMoment_1_0',
       'StDev_ICM_AreaShape_NormalizedMoment_0_0',
       'StDev_ICM_AreaShape_NormalizedMoment_0_1',
       'StDev_ICM_AreaShape_NormalizedMoment_1_0',
       'StDev_Outlines_AreaShape_NormalizedMoment_0_0',
       'StDev_Outlines_AreaShape_NormalizedMoment_0_1',
       'StDev_Outlines_AreaShape_NormalizedMoment_1_0'], inplace=True)

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from scipy.stats import randint

#load training and testing data
train_df = aligned_df[aligned_df['category'] == 'Silver'].drop(columns='category')
test_df = aligned_df[aligned_df['category'] == 'Silver'].drop(columns='category')

#create train, test objects
X_train = train_df.drop(columns=['FileName_BlastocystOriginal', 'ImageSet_ImageSet',
       'MD5Digest_BlastocystOriginal', 'Metadata_FileLocation',
       'PathName_BlastocystOriginal', 'ProcessingStatus',
       'URL_BlastocystOriginal', 'label'])
y_train = train_df['label']

X_test = test_df.drop(columns=['FileName_BlastocystOriginal', 'ImageSet_ImageSet',
       'MD5Digest_BlastocystOriginal', 'Metadata_FileLocation',
       'PathName_BlastocystOriginal', 'ProcessingStatus',
       'URL_BlastocystOriginal', 'label'])
y_test = test_df['label']

#Implement RF model
model = RandomForestClassifier()

#Define the parameter distribution
param_dist = {
    'n_estimators': randint(10, 200),        
    'max_features': ['auto', 'sqrt', 'log2'],  
    'max_depth': randint(1, 20),               
    'min_samples_split': randint(2, 20),      
    'min_samples_leaf': randint(1, 20)        
}

#Create the RandomizedSearchCV object to optimize for best features
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=5, verbose=2, scoring='accuracy', random_state=42, error_score='raise')

#Perform randomized search
random_search.fit(X_train, y_train)

#Output best parameters and corresponding score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

#Use the best model to make predictions
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

#Evaluate the model
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END max_depth=7, max_features=auto, min_samples_leaf=15, min_samples_split=12, n_estimators=81; total time=   1.2s


/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END max_depth=7, max_features=auto, min_samples_leaf=15, min_samples_split=12, n_estimators=81; total time=   1.2s


...

[CV] END max_depth=19, max_features=auto, min_samples_leaf=4, min_samples_split=13, n_estimators=198; total time=   4.3s


/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END max_depth=19, max_features=auto, min_samples_leaf=4, min_samples_split=13, n_estimators=198; total time=   4.3s
[CV] END max_depth=9, max_features=log2, min_samples_leaf=12, min_samples_split=10, n_estimators=37; total time=   0.2s
[CV] END max_depth=9, max_features=log2, min_samples_leaf=12, min_samples_split=10, n_estimators=37; total time=   0.2s
[CV] END max_depth=9, max_features=log2, min_samples_leaf=12, min_samples_split=10, n_estimators=37; total time=   0.2s
[CV] END max_depth=9, max_features=log2, min_samples_leaf=12, min_samples_split=10, n_estimators=37; total time=   0.2s
[CV] END max_depth=9, max_features=log2, min_samples_leaf=12, min_samples_split=10, n_estimators=37; total time=   0.2s
[CV] END max_depth=14, max_features=log2, min_samples_leaf=19, min_samples_split=16, n_estimators=153; total time=   0.8s
[CV] END max_depth=14, max_features=log2, min_samples_leaf=19, min_samples_split=16, n_estimators=153; total time=   0.8s
[CV] END max_depth=14, max_features

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Parameters: {'max_depth': 13, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 164}
Best Score: 0.7240699458267414
              precision    recall  f1-score   support

        Fair       1.00      0.84      0.91       364
       Great       0.95      1.00      0.97      1289
        Poor       0.99      0.96      0.98       391

    accuracy                           0.96      2044
   macro avg       0.98      0.93      0.95      2044
weighted avg       0.97      0.96      0.96      2044

